# Getting starting fine-tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B.

<a target="_blank" href="https://colab.research.google.com/github/hadiaskari1981/mistral-finetune-hadi/blob/main/mistral_viggo_fine_tune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Let's begin!
The libraries to install. Run once per instance

In [ ]:
!pip install bitsandbytes
!pip install datasets==2.17.1
!pip install accelerate
!pip install peft
!pip install transformers
!pip install scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from datetime import datetime
import transformers
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

In [ ]:
# the config for peft_model
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

## Data preprocessing

- Load the data
- Get the tokenizer from the model
- Define the prompt
- Generate the tokenized propmt for train and val datasets

In [ ]:
class DataTokenizer:
    def __init__(self, args: dict):
        self.data_path = "gem/viggo"
        self.model_max_length = args["model_max_length"]
        self.base_model_name = args["base_model_name"]
        self.cache_dir = args.get("cache_dir", None)

        self.tokenizer = AutoTokenizer.from_pretrained(
            self.base_model_name,
            model_max_length=512,
            padding_side="left",
            add_eos_token=True,
            cache_dir=self.cache_dir)

        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.train_dataset = load_dataset(
            self.data_path, split='train', trust_remote_code=True, cache_dir=self.cache_dir
        )
        self.val_dataset = load_dataset(
            self.data_path, split='validation', trust_remote_code=True, cache_dir=self.cache_dir
        )
        self.test_dataset = load_dataset(
            self.data_path, split='test', trust_remote_code=True, cache_dir=self.cache_dir
        )

        self.tokenized_train_dataset = self.train_dataset.map(self.generate_and_tokenize_prompt)
        self.tokenized_val_dataset = self.val_dataset.map(self.generate_and_tokenize_prompt)

    def prompt_tokenizer(self, prompt):
        result = self.tokenizer(
            prompt,
            truncation=True,
            max_length=512,
            padding="max_length",
        )
        result["labels"] = result["input_ids"].copy()
        return result

    def generate_and_tokenize_prompt(self, data_point):
        full_prompt = f"""Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
                    This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
                    The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

                    ### Target sentence:
                    {data_point["target"]}

                    ### Meaning representation:
                    {data_point["meaning_representation"]}
                    """
        return self.prompt_tokenizer(full_prompt)

## Load the base model either for training or inference

In [ ]:
class BaseModelLoader:
    def __init__(self, args):
        self.base_model_name = args["base_model_name"]
        self.cache_dir = args.get("cache_dir", None)
        self.is_inference = args.get("is_inference")
        self.quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        if self.is_inference:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.base_model_name,
                quantization_config=self.quantization_config,
                cache_dir=self.cache_dir,
                device_map="auto",
                trust_remote_code=True
            )
        else:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.base_model_name,
                quantization_config=self.quantization_config,
                cache_dir=self.cache_dir)

Creates an instance of an accelerator for distributed training (on multi-GPU, TPU) or mixed precision training.</pre>
I am not sure if it is helpful when using only one GPU.

In [ ]:
def accelerator():
    plugin = FullyShardedDataParallelPlugin(
        state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
        optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
    )

    acc = Accelerator(fsdp_plugin=plugin)

    return acc

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
### create the path to save model, data, and output of the code

In [ ]:
%cd /content/drive/MyDrive/GitHub/mistral-finetune-hadi

/content/drive/MyDrive/GitHub/mistral-finetune-hadi


In [ ]:
os.getcwd()

'/content/drive/MyDrive/GitHub/mistral-finetune-hadi'

In [ ]:
project = "mistral-viggo-finetune"

current_working_dir = os.getcwd()

saved_dir = f"{current_working_dir}/data"
os.makedirs(saved_dir, exist_ok=True)

cache_path = f'{saved_dir}/cache'
os.makedirs(cache_path, exist_ok=True)

model_output_path = f"{saved_dir}/{project}"
os.makedirs(model_output_path, exist_ok=True)

logs_path = f"{saved_dir}/logs"
os.makedirs(logs_path, exist_ok=True)

In [ ]:
### arguments to use in the training

In [ ]:
train_args = {
    "base_model_name": "mistralai/Mistral-7B-v0.3",
    "cache_dir": cache_path,
    "is_inference": False,
    "data_path": "gem/viggo",
    "model_max_length": 512,
    "model_output_dir": model_output_path,
    "logs_dir": logs_path,
    "project_name": project
}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
### load base_model for the training

In [ ]:
base_model = BaseModelLoader(train_args).model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
### tokenized the train and val datasets

In [ ]:
data_tokenizer = DataTokenizer(train_args)

tokenized_train_dataset = data_tokenizer.tokenized_train_dataset
tokenized_val_dataset = data_tokenizer.tokenized_val_dataset

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating challenge_train_5_percent split: 0 examples [00:00, ? examples/s]

Generating challenge_train_10_percent split: 0 examples [00:00, ? examples/s]

Generating challenge_train_20_percent split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5103 [00:00<?, ? examples/s]

Map:   0%|          | 0/714 [00:00<?, ? examples/s]

To start fine-tuning, we should pre-process the model. For that, we use the `prepare_model_for_kbit_training` from peft.

In [ ]:
base_model.gradient_checkpointing_enable()
base_model = prepare_model_for_kbit_training(base_model)

In [ ]:
### the config contains the paramaters to define low-rank matrix

In [ ]:
base_model = get_peft_model(base_model, config)
print_trainable_parameters(base_model)

trainable params: 21266432 || all params: 3779629056 || trainable%: 0.5626592367904582


In [ ]:
### Apply the accelerator. You can comment this out to remove the accelerator.

In [ ]:
acc = accelerator()
base_model = acc.prepare_model(base_model)

In [ ]:
tokenizer = data_tokenizer.tokenizer

trainer = transformers.Trainer(
    model=base_model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
    output_dir=train_args.get("model_output_dir"),
    warmup_steps=5,
    per_device_train_batch_size=2,
    gradient_checkpointing=True,
    gradient_accumulation_steps=4,
    max_steps=100,
    learning_rate=2.5e-5,  # Want about 10x smaller than the Mistral learning rate
    logging_steps=50,
    bf16=True,
    optim="paged_adamw_8bit",
    logging_dir=train_args.get("logs_dir"),  # Directory for storing logs
    save_strategy="steps",  # Save the model checkpoint every logging step
    save_steps=10,  # Save checkpoints every 50 steps
        evaluation_strategy="steps",  # Evaluate the model every logging step
    eval_steps=10,  # Evaluate and save checkpoints every 10 steps
    do_eval=True,  # Perform evaluation at the end of training
    run_name=f"{train_args.get('project_name')}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


### Train

In [ ]:
base_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

### Let's try the trained model

In [ ]:
inference_args = {
    "base_model_name": "mistralai/Mistral-7B-v0.3",
    "cache_dir": cache_path,
    "is_inference": True,
    "data_path": "gem/viggo",
    "model_max_length": 512,
    "project_name": project,
    "model_output_path": model_output_path,
    "checkpoint": "checkpoint-1000",
    "target_sentence": "I remember you saying you found Little Big Adventure to be average. Are you not usually "
                       "that into single-player games on PlayStation?"
}

In [ ]:
### load base-model for inference

In [ ]:
base_model = BaseModelLoader(inference_args).model

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(
        inference_args.get("base_model_name"),
        add_bos_token=True,
        trust_remote_code=True,
        cache_dir=inference_args.get("cache_dir)
    )

In [ ]:
### load fine-tuned model

In [ ]:
output_dir = inference_args.get("model_output_path")
checkpoint_version = inference_args.get("checkpoint")
ft_model = PeftModel.from_pretrained(base_model, f"{output_dir}/{checkpoint_version}")

In [ ]:
eval_prompt = f"""Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
    This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
    The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

    ### Target sentence:
    {inference_args.get("target_sentence")}

    ### Meaning representation:
    """

In [ ]:
### fine-tuned model outcome

In [ ]:
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))

In [ ]:
### base model outcome

In [ ]:
base_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))